# MultiModal Kinetic Modeling

In this tutorial we will investigate how to simultaneously optimize transient optical and transient X-ray emission spectroscopy 

In [1]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)
%matplotlib qt

Plot_func version 7.12.5
was imported from path:
 C:\Users\Jens\anaconda3\Lib\site-packages\KiMoPack
The current working folder is:
 d:\Dropbox\coding\github\KiMoPack\Tutorial_Notebooks
Plot_func version 7.12.5
was imported from path:
 C:\Users\Jens\anaconda3\Lib\site-packages\KiMoPack
The current working folder is:
 d:\Dropbox\coding\github\KiMoPack\Tutorial_Notebooks


## 1) Load Data


In [2]:
filepath = os.path.join(os.getcwd(), 'Data', 'MultiModal')  # set path to file to fit

Prepare the transient optical data

In [3]:
ta=pf.TA(filename='combined_optical_spectrum.SIA',path=filepath)  #load transient optical
plt.close('all')
#ta.Background()
ta.wavelength_bin=20                                        # width of wavelength bin in kinetics
ta.wave_nm_bin=5;
ta.ignore_time_region
ta.rel_wave=np.concatenate([np.arange(300,572+25,25),np.arange(662,800,25)])# rebinning wavelength
ta.timelimits=[-0.2,100]                                     # use to manually limit the range to plot (and fit)     
ta.bordercut=[335,766]                                      # use to set the outer wavelength limits
ta.scattercut=[572,662]                                     # this region is blanked out to block scatter, this can be a list of regions
ta.intensity_range=1.2e-2                          # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
ta.Plot_RAW([0,3])                                      # Plot Raw Spectra

c:\Users\Jens\anaconda3\Lib\site-packages\KiMoPack\plot_func.py:5499: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if (self.ds_ori.applymap(type) == float).all().all():


In [4]:
import function_library as func

In [5]:
plt.close('all')                                             # Close old spectra
ta.mod=func.P12                                        # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/0.275,vary=True)                                # Add second rate component
par.add('k1',value=1/7.2,vary=True)                                # Add one rate component
par.add('k2',value=1/3,vary=True)                                 # Add second rate component
par.add('t0',value=-0.0167,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=0.0935,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (for all)
#par.add('explicit_GS')
par.add('infinite')                                                # Keyword for an non decaying component
#par.add('background')                                              # Keyword for an non decaying component                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
ta.ignore_time_region=[-0.5,0.3]                                      # select time window in which to fit
ta.log_fit=False                                               # fit in linear or log scale
ta.timelimits=[-1,200]
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted       
ta.log_scale=False
ta.Plot_fit_output([0,4])# 
#ta.Plot_fit_output(range(5))                                 # plot the fit output

Optimizing, after the starting error the new error values will be displayed every 10s 
 to interrupt try to press ctrl+shift+q
0.0009240314078486792

Fit Results:
Model Used: External function

the time between -0.500 ps and 0.300 ps was excluded from the optimization

The minimum error is:5.21974521e-04
The minimum R2-value is:9.93848532e-01

In Rates

+------------+-----------+--------------+--------+---------+-------+--------+
|            |     value |   init_value | vary   |     min |   max | expr   |
+============+===========+==============+========+=========+=======+========+
| k0         |  1.00094  |     3.63636  | True   |    0    | inf   |        |
+------------+-----------+--------------+--------+---------+-------+--------+
| k1         |  0.141746 |     0.138889 | True   |    0    | inf   |        |
+------------+-----------+--------------+--------+---------+-------+--------+
| k2         |  0.488891 |     0.333333 | True   |    0    | inf   |        |
+------------+------

In [ ]:
ta.Save_Powerpoint()

These results are terrible, mainly because the optical artifacts in the data are disturbing the system. Lets see what the XES is doing.

In [6]:
xes=pf.TA('XES_on.SIA',path=filepath,sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission intensity',baseunit='ps')   #
plt.close('all')
xes.ds=pf.Frame_golay(xes.ds,7,1)
xes.rel_wave=np.arange(7053,7061,1)      # use to manually select interesting wavelength for the kinetics
xes.wavelength_bin=1                                        # width of wavelength bin in kinetics
xes.rel_time=[-0.5,-0.1,0,0.1,0.3,0.5,1,1.5,3,5,10,15,20]      # use to manually set interesting times for the spectra
xes.time_width_percent=0                                    # rebinning of time_points in percent
xes.timelimits=[-1,200]                                     # use to manually limit the range to plot (and fit)     
xes.log_scale=False                                         # use to plot the 2d plots with logarithmic intensity scale
xes.bordercut=[7045,7065]                                      # use to set the outer wavelength limits
xes.intensity_range=[0,7]                           # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
#xes.Plot_RAW()                                      # Plot Raw Spectra

In [7]:
plt.close('all')                                            
xes.mod=func.P12                                       
par=ta.par_fit                                       
par['t0'].value=-0.0167                               
par['resolution'].value=0.0935
#par.add('explicit_GS')
par.add('infinite')                                                # Keyword for an non decaying component
par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect

if 0:
    for key in par.keys():
        par[key].vary=False
xes.par=par                                                     # write parameter object into file for fitting
xes.Fit_Global()                                 # trigger fitting
xes.error_matrix_amplification=1;                              # Choose how much stronger should error be plotted                                                    # 2D plots in linear or log scale
xes.Plot_fit_output([0,4])                                 # plot the fit output

Optimizing, after the starting error the new error values will be displayed every 10s 
 to interrupt try to press ctrl+shift+q
63.19231553376781

Fit Results:
Model Used: External function

The minimum error is:5.75664126e+01
The minimum R2-value is:9.98866793e-01

In Rates

+------------+------------+--------------+--------+---------+-------+--------+
|            |      value |   init_value | vary   |     min |   max | expr   |
+============+============+==============+========+=========+=======+========+
| k0         |  4.90226   |     1.00094  | True   |    0    | inf   |        |
+------------+------------+--------------+--------+---------+-------+--------+
| k1         |  0.200664  |     0.141746 | True   |    0    | inf   |        |
+------------+------------+--------------+--------+---------+-------+--------+
| k2         |  0.0131696 |     0.488891 | True   |    0    | inf   |        |
+------------+------------+--------------+--------+---------+-------+--------+
| t0         

In [ ]:
xes.Save_Powerpoint()

In [8]:
xes.Background(uplimit=-0.25)
xes.intensity_range=1
try:
    xes.par.pop('background')
except:
    pass
xes.Fit_Global(confidence_level=0.95)                                 # trigger fitting
plt.close('all')
xes.Plot_fit_output([0,4])

Optimizing, after the starting error the new error values will be displayed every 10s 
 to interrupt try to press ctrl+shift+q
64.88585719479536
Trying to find k0, lower confidence limit
59.35474713433317
59.36541824015963
Trying to find k0, upper confidence limit
59.45208641214517
61.33518859573539
60.981044576714226
60.646073626373244
60.65430712574518
Trying to find k1, lower confidence limit
59.691904161560124
60.64510738460195
Trying to find k1, upper confidence limit
59.42894513938775
61.104687822982164
Trying to find k2, lower confidence limit
59.35472848763973
59.35564407316684
59.362043855121456
Trying to find k2, upper confidence limit
59.35765512016757
59.35950209330571
it took 207 optimisations to get the confidence

Fit Results:
Model Used: External function

The minimum error is:5.93547284e+01
The minimum R2-value is:9.81344648e-01

In Rates with confidence interval to level of 95.0

+------------+------------+---------------+---------------+--------------+--------+------

In [ ]:
xes.Save_Powerpoint()

In [9]:
weights=[xes.re['A'].abs().sum().sum(),ta.re['A'].abs().sum().sum()]
print(weights)
ta1=ta.Copy()
xes1=xes.Copy()

[4534.596636809537, 17.334058276781647]


In [10]:
for key in ['k0','k1','k2']:
	xes1.par[key].vary=True

In [11]:
xes1.Fit_Global(multi_project=[ta1],same_DAS=False,same_shape_params=False,weights=weights,unique_parameter=['resolution','t0'],confidence_level=0.95)  # trigger fitting
xes1.Plot_fit_output()

Optimizing, after the starting error the new error values will be displayed every 10s 
 to interrupt try to press ctrl+shift+q
208052.8695527355
207485.7212882863
205333.92864390637
207739.68013499383
205567.45381697928
204585.61417626933
203281.41113005165
202626.73911296917
201209.77030858138
201438.84532060055
199772.83923074178
198833.51778797797
199589.32983966067
199042.22479136282
200238.35542873634
199386.4359335701
198536.32015880255
198257.8749508627
197951.50482594248
197687.69469328932
197039.40180706495
196349.5329507435
195568.0739375222
194373.4545012273
196826.63763932505
193283.69981002805
192031.35279951637
193807.4739290231
192073.2675203987
190855.33938566683
190962.3345140406
190612.16365852227
193545.6897612543
193262.48257309984
190950.946556311
191732.55659462122
190771.06099403015
190436.00437740743
191001.45979353297
191003.31179969225
190463.85279692948
190999.3348488516
190421.75908904523
191354.5227567155
190339.9114853685
190533.80332904487
190365.27529152

In [12]:
xes2=xes.Copy()
weights=[1/xes.re['A'].abs().sum().sum(),1/ta.re['A'].abs().sum().sum()]
xes2.Fit_Global(multi_project=[ta1],same_DAS=False,same_shape_params=False,weights=weights,unique_parameter=['resolution','t0'],confidence_level=0.95)  # trigger fitting
xes2.Plot_fit_output()

Optimizing, after the starting error the new error values will be displayed every 10s 
 to interrupt try to press ctrl+shift+q
0.010133481576583276
0.010105168801313187
0.010000581574990827
0.010117842114732782
0.010011427916521095
0.00996354572209618
0.009899682615408757
0.009867942927713223
0.00979862050511216
0.00981000669024617
0.009727834785081527
0.0096812023811669
0.009718494608327626
0.009690994119903387
0.009748198956450634
0.009707400645786773
0.009665615436917517
0.009651417661583459
0.00963746860496662
0.009624614121845104
0.009592500604256236
0.009558544888991346
0.00951979242006007
0.009461011418905015
0.009581300569311002
0.009407862678350917
0.009346714937568931
0.009432926062989724
0.009348772085018945
0.009290088899935019
0.009297182923405747
0.009278908591660402
0.009423371501178047
0.00940742729281368
0.009294348789666526
0.00933477731072978
0.009286865475113851
0.009271498959075293
0.009300701789379391
0.009299711709681009
0.009271541180288556
0.009298092280018171


In [ ]:
xes2.filename='XES_with_TA_refined'
xes2.Save_Powerpoint()

In [ ]:
xes1.par=ta1.par_fit
for key in xes1.par.keys():
    xes1.par[key].vary=False
xes1.Fit_Global()
plt.close('all')
xes1.Plot_fit_output()